In [1]:
import skeleton as s
import numpy as np

In [2]:
init_pos = np.random.uniform(low=0, high=1.0, size=(2, 2))
init_vel = np.random.uniform(low=-430, high=430, size=(2, 2))
time_step = 0.1
m = 6.6335 * pow(10, -26) #mass of argon atom in kg
n = 2 #number of particles
number_of_steps = 100

In [3]:
print("initial pos\n", init_pos,"\n initial vel\n" ,init_vel)

initial pos
 [[0.56749271 0.17444576]
 [0.72246894 0.87556052]] 
 initial vel
 [[ -63.39062695 -301.8077322 ]
 [ -50.50007198   30.33100338]]


In [4]:
#compute the forces on the particles at each timestep
def lj_force(position, n):
    e = 1.65 *pow(10,-21)
    s = 3.405 *pow(10,-21)
    F = 0
    pos = position[:,-n:]
    for j in range(n):
        f = 0    # force on the j-th particle
        for i in range(n-1):
            if i+j+1 >= n: #take into account particles 0,1,....,j-1
                rel_pos=pos[:,j+i+1-n]-pos[:,j] #gives position of i-th particle with resprect to j-th particle (points from i to j)
                r = np.linalg.norm(pos[:,j+i+1-n]-pos[:,j])
                f += rel_pos*((24*e/pow(r,2))*(pow(s/r,6))*((pow(s/r,6))-1))  # ff on the j-th part. from the interaction with the i-th part.
            else: #take into account particles j+1,....,n-1
                rel_pos=pos[:,j+i+1]-pos[:,j] #gives position of i-th particle with resprect to j-th particle (points from i to j)
                r = np.linalg.norm(pos[:,j+i+1]-pos[:,j])
                f += rel_pos*((24*e/pow(r,2))*(pow(s/r,6))*((pow(s/r,6))-1))  # f on the j-th part. from the interaction with the i-th part.
        if j == 0:
            F = f
        else:
            F = np.concatenate((F, f), axis=0, out=None)
    
    F_matrix = np.zeros((2,n)) #2 is the number of dimensions, n is the number of particles
    F_matrix[:n] = F[0:n]
    F_matrix[1:] = F[2:]
    return F_matrix

In [5]:
final_matrix_pos = np.copy(init_pos)
final_matrix_vel = np.copy(init_vel)

def euler(final_matrix_pos, final_matrix_vel):
    
    latest_pos = np.copy(final_matrix_pos[:,-2:])          #the -2 will eventually become -(number of particles)
    latest_vel = np.copy(final_matrix_vel[:,-2:])
    
    new_latest_pos = latest_pos + latest_vel * time_step
    new_latest_vel = latest_vel + 1/m * lj_force(latest_pos, 2)
    return new_latest_pos, new_latest_vel


In [6]:
def kin_en(v,n): #v is the last step velocity and n the number of particles
    K = 0
    for i in range(n):
        K += 0.5*m*((v[0,i]*v[0,i])+(v[1,i]*v[1,i]))
    return K

In [7]:
e = 1.65*pow(10,-21)
s = 3.405*pow(10,-10)
def pot_en(position, n): #position is the matrix with all the positions stored in it, n is the number of particles
    U = 0;
    pos = position[:,-n:]
    for j in range(n):
        for i in range(n-1):
            if i+j+1 >= n: #take into account particles 0,1,....,j-1
                r = np.linalg.norm(pos[:,j+i+1-n]-pos[:,j])
                U += (4*e)*(pow(s/r,6))*((pow(s/r,6))-1)  # potnetial energy of the j-th part. from the interaction with the i-th part.
            else: #take into account particles j+1,....,n-1
                r = np.linalg.norm(pos[:,j+i+1]-pos[:,j])
                U += (4*e)*(pow(s/r,6))*((pow(s/r,6))-1)  # potential energy of the j-th part. from the interaction with the i-th part.
                
    return U

In [8]:
#Create a 2x8 matrix to store the velocity of each particle at each step in time.
next_step_velocity = np.copy(init_vel)

#Create a 2x8 matrix to store the position of each particle at each step in time.
next_step_position = np.copy(init_pos)

final_vector_energy = np.array([kin_en(init_vel, n) + pot_en(final_matrix_pos, n)])

for i in range(number_of_steps):
    next_step_position, next_step_velocity = euler(final_matrix_pos, final_matrix_vel)
    for k in range(2): #implement boundary conditions
        for j in range(2):
            next_step_position[k, j] = next_step_position[k, j]%1
    final_matrix_pos =  np.concatenate((final_matrix_pos, next_step_position), axis=1, out=None)
    final_matrix_vel =  np.concatenate((final_matrix_vel, next_step_velocity), axis=1, out=None)
    final_vector_energy = np.concatenate((final_vector_energy, np.array([kin_en(next_step_velocity, n) + pot_en(final_matrix_pos, n)])), axis=0, out=None)

print("Positions:\n" , final_matrix_pos)
print("Velocities:\n" , final_matrix_vel)
print("Energy:\n" , final_vector_energy)

Positions:
 [[5.67492711e-01 1.74445762e-01 2.28430016e-01 9.93672542e-01
  8.89367320e-01 8.12899321e-01 5.50304625e-01 6.32126101e-01
  2.11241930e-01 4.51352880e-01 8.72179235e-01 2.70579660e-01
  5.33116540e-01 8.98064391e-02 1.94053844e-01 9.09033219e-01
  8.54991149e-01 7.28259998e-01 5.15928454e-01 5.47486778e-01
  1.76865759e-01 3.66713557e-01 8.37803064e-01 1.85940337e-01
  4.98740368e-01 5.16711614e-03 1.59677673e-01 8.24393896e-01
  8.20614978e-01 6.43620675e-01 4.81552283e-01 4.62847455e-01
  1.42489588e-01 2.82074234e-01 8.03426892e-01 1.01301014e-01
  4.64364197e-01 9.20527793e-01 1.25301502e-01 7.39754573e-01
  7.86238807e-01 5.58981352e-01 4.47176112e-01 3.78208132e-01
  1.08113417e-01 1.97434911e-01 7.69050721e-01 1.66616908e-02
  4.29988026e-01 8.35888470e-01 9.09253309e-02 6.55115250e-01
  7.51862636e-01 4.74342029e-01 4.12799941e-01 2.93568809e-01
  7.37372453e-02 1.12795588e-01 7.34674550e-01 9.32022368e-01
  3.95611855e-01 7.51249147e-01 5.65491598e-02 5.70475927e